#### Projeto - We Rate Dog - Twitter
#### Curso - Fundamentos de Data Science II

#### Aluno Leandro Baruch 

## Motivação do projeto
#### Meta
Meta do Projeto: fazer wrangling dos dados de tweets de WeRateDogs para que você possamos criar análises e visualizações interessantes e confiáveis. Coleta, avaliação e limpeza adicionais são necessárias para análises e visualizações que mereçam uma reação de "Uau!"


## Coleta

In [1]:
#Importando as bibliotecas
import pandas as pd
import requests as req
import os
import numpy as np
import json
import io

In [2]:
#Importando as APIs
import tweepy

consumer_key = 'zf65NofmWv0IDjWQTCra3HpJb'
consumer_secret = 'ZxxxyhMDL18GwaDwQivOrAIv4Q9N8FSFyYeQBErfsx4tsIAB7q'
access_token = '1061992594450927616-0PpjeNAXiRu6nIKoSv4CtVKX2vvGFe'
access_secret = '5S7aSleOeWyPfWVFKD3xM2RfrtCwG8jG5k2rMz0DDTFre'

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)
api = tweepy.API(auth, wait_on_rate_limit=True,wait_on_rate_limit_notify=True)

In [3]:
#Criando o dataframe do arquivo twitter-archive-enhanced.csv
twitter_archive = pd.read_csv('twitter-archive-enhanced.csv')
twitter_archive.head()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None


In [4]:
# Criação da pasta 'image_prediction'
folder_name = 'image_prediction'
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

In [5]:
# Download do arquivo image-predictions.tsv
image_prediction_url = ['https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv']
for url in image_prediction_url:
    response = req.get(url)
    with open(os.path.join(folder_name, url.split('/')[-1]), mode = 'wb') as file:
        file.write(response.content)

In [6]:
#Criando o dataframe do arquivo image_predictions.tsv
image_predictions = pd.read_csv('image_prediction/image-predictions.tsv', sep='\t')
image_predictions.head()

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True


In [7]:
#Criando uma lista com os IDs e o arquivo tweet_json com os status dos Tweets
tweet_ids = list(twitter_archive.tweet_id)
tweet_file = open("tweet_json.txt","w")

for tweet in tweet_ids:
    try:
        tweet_status = api.get_status(tweet, tweet_mode='extended')
        tweet_file.write(json.dumps(tweet_status._json)) # store tweets in json format
        tweet_file.write('\n')
    except:
        print("Error for: " + str(tweet))
        
tweet_file.close()

Error for: 888202515573088257
Error for: 873697596434513921
Error for: 872668790621863937
Error for: 869988702071779329
Error for: 866816280283807744
Error for: 861769973181624320
Error for: 845459076796616705
Error for: 842892208864923648
Error for: 837012587749474308
Error for: 827228250799742977
Error for: 802247111496568832
Error for: 775096608509886464
Error for: 770743923962707968


Rate limit reached. Sleeping for: 700


Error for: 754011816964026368


Rate limit reached. Sleeping for: 704


In [8]:
#Criar uma lista com as informações do arquivo tweet_json
tweet_list =[]
json_file = open("tweet_json.txt", "r")
for line in json_file:
    tweets = json.loads(line) # load json file
    tweet_id = tweets['id']
    favorite_counts = tweets['favorite_count']
    retweet_counts = tweets['retweet_count']
    tweet_list.append({'tweet_id': int(tweet_id), # create a list with the columns
                    'favorite_counts' : int(favorite_counts),
                    'retweet_counts': int(retweet_counts)})



In [9]:
# Converter a lista para um Dataframe
tweet_status = pd.DataFrame(tweet_list, columns = ['tweet_id', 'favorite_counts' , 'retweet_counts']) 
tweet_status.head()

,tweet_id,favorite_counts,retweet_counts
0,892420643555336193,38211,8371
1,892177421306343426,32758,6179
2,891815181378084864,24666,4091
3,891689557279858688,41542,8506
4,891327558926688256,39727,9212


# Análise

In [10]:
#Analisando o dataframe twitter archive
twitter_archive.head(20)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None
5,891087950875897856,NaN,NaN,2017-07-29 00:08:17 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here we have a majestic great white breaching ...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891087950...,13,10,None,None,None,None,None
6,890971913173991426,NaN,NaN,2017-07-28 16:27:12 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Meet Jax. He enjoys ice cream so much he gets ...,NaN,NaN,NaN,"https://gofundme.com/ydvmve-surgery-for-jax,ht...",13,10,Jax,None,None,None,None
7,890729181411237888,NaN,NaN,2017-07-28 00:22:40 +0000,"<a href=""http://twitter.com/download/iphone"" r...",When you watch your owner call another dog a g...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/890729181...,13,10,None,None,None,None,None
8,890609185150312448,NaN,NaN,2017-07-27 16:25:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Zoey. She doesn't want to be one of th...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/890609185...,13,10,Zoey,None,None,None,None
9,890240255349198849,NaN,NaN,2017-07-26 15:59:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Cassie. She is a college pup. Studying...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/890240255...,14,10,Cassie,doggo,None,None,None


In [11]:
twitter_archive.sample(15)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
2228,668256321989451776,NaN,NaN,2015-11-22 02:34:57 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Jareld. Jareld rules these waters. Lad...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/668256321...,13,10,Jareld,None,None,None,None
2237,668142349051129856,NaN,NaN,2015-11-21 19:02:04 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This lil pup is Oliver. Hops around. Has wings...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/668142349...,2,10,None,None,None,None,None
1055,742385895052087300,NaN,NaN,2016-06-13 15:59:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phil. That's his comfort stick. He hol...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/742385895...,11,10,Phil,None,None,None,None
566,802247111496568832,NaN,NaN,2016-11-25 20:26:31 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @dog_rates: Everybody drop what you're doin...,7.790561e+17,4.196984e+09,2016-09-22 20:33:42 +0000,https://twitter.com/dog_rates/status/779056095...,13,10,None,None,None,None,None
1081,738885046782832640,NaN,NaN,2016-06-04 00:08:17 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Charles. He's a Nova Scotian Towel Pou...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/738885046...,11,10,Charles,None,None,None,None
1832,676191832485810177,NaN,NaN,2015-12-14 00:07:50 +0000,"<a href=""http://twitter.com/download/iphone"" r...",These two pups just met and have instantly bon...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/676191832...,10,10,None,None,None,None,None
374,828372645993398273,NaN,NaN,2017-02-05 22:40:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Alexander Hamilpup. He was one of the ...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/828372645...,12,10,Alexander,None,None,None,None
1315,706681918348251136,NaN,NaN,2016-03-07 03:24:33 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Say hello to Katie. She's a Mitsubishi Hufflep...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/706681918...,12,10,Katie,None,None,None,None
418,822462944365645825,NaN,NaN,2017-01-20 15:17:01 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Gabe. He was the unequivocal embodimen...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/822462944...,14,10,Gabe,None,None,pupper,None
2121,670408998013820928,NaN,NaN,2015-11-28 01:08:55 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Peanut. He was the World Table Tennis ...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/670408998...,10,10,Peanut,None,None,None,None


In [12]:
twitter_archive.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

In [13]:
twitter_archive[twitter_archive.retweeted_status_id.notnull() == True]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
19,888202515573088257,NaN,NaN,2017-07-21 01:02:36 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @dog_rates: This is Canela. She attempted s...,8.874740e+17,4.196984e+09,2017-07-19 00:47:34 +0000,https://twitter.com/dog_rates/status/887473957...,13,10,Canela,None,None,None,None
32,886054160059072513,NaN,NaN,2017-07-15 02:45:48 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @Athletics: 12/10 #BATP https://t.co/WxwJmv...,8.860537e+17,1.960740e+07,2017-07-15 02:44:07 +0000,https://twitter.com/dog_rates/status/886053434...,12,10,None,None,None,None,None
36,885311592912609280,NaN,NaN,2017-07-13 01:35:06 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @dog_rates: This is Lilly. She just paralle...,8.305833e+17,4.196984e+09,2017-02-12 01:04:29 +0000,https://twitter.com/dog_rates/status/830583320...,13,10,Lilly,None,None,None,None
68,879130579576475649,NaN,NaN,2017-06-26 00:13:58 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @dog_rates: This is Emmy. She was adopted t...,8.780576e+17,4.196984e+09,2017-06-23 01:10:23 +0000,https://twitter.com/dog_rates/status/878057613...,14,10,Emmy,None,None,None,None
73,878404777348136964,NaN,NaN,2017-06-24 00:09:53 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @dog_rates: Meet Shadow. In an attempt to r...,8.782815e+17,4.196984e+09,2017-06-23 16:00:04 +0000,"https://www.gofundme.com/3yd6y1c,https://twitt...",13,10,Shadow,None,None,None,None
74,878316110768087041,NaN,NaN,2017-06-23 18:17:33 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @dog_rates: Meet Terrance. He's being yelle...,6.690004e+17,4.196984e+09,2015-11-24 03:51:38 +0000,https://twitter.com/dog_rates/status/669000397...,11,10,Terrance,None,None,None,None
78,877611172832227328,NaN,NaN,2017-06-21 19:36:23 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @rachel2195: @dog_rates the boyfriend and h...,8.768508e+17,5.128045e+08,2017-06-19 17:14:49 +0000,https://twitter.com/rachel2195/status/87685077...,14,10,None,None,None,pupper,None
91,874434818259525634,NaN,NaN,2017-06-13 01:14:41 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @dog_rates: This is Coco. At first I though...,8.663350e+17,4.196984e+09,2017-05-21 16:48:45 +0000,https://twitter.com/dog_rates/status/866334964...,12,10,Coco,None,None,None,None
95,873697596434513921,NaN,NaN,2017-06-11 00:25:14 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @dog_rates: This is Walter. He won't start ...,8.688804e+17,4.196984e+09,2017-05-28 17:23:24 +0000,https://twitter.com/dog_rates/status/868880397...,14,10,Walter,None,None,None,None
97,873337748698140672,NaN,NaN,2017-06-10 00:35:19 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @dog_rates: This is Sierra. She's one preci...,8.732138e+17,4.196984e+09,2017-06-09 16:22:42 +0000,https://www.gofundme.com/help-my-baby-sierra-g...,12,10,Sierra,None,None,pupper,None


In [14]:
twitter_archive['name'].value_counts()

None         745
a             55
Charlie       12
Lucy          11
Cooper        11
Oliver        11
Lola          10
Penny         10
Tucker        10
Winston        9
Bo             9
Sadie          8
the            8
Buddy          7
an             7
Bailey         7
Daisy          7
Toby           7
Jack           6
Bella          6
Stanley        6
Koda           6
Rusty          6
Milo           6
Oscar          6
Scout          6
Jax            6
Leo            6
Dave           6
Larry          5
            ... 
Kellogg        1
Augie          1
Ralphson       1
Alexander      1
Duchess        1
Ed             1
Luther         1
Nida           1
Banditt        1
Doobert        1
Apollo         1
Nugget         1
Buddah         1
Dug            1
Lili           1
Comet          1
Dewey          1
Kanu           1
Tino           1
life           1
Shikha         1
Taz            1
Huxley         1
Terrenth       1
Reptar         1
Shelby         1
Linus          1
Snoopy        

In [15]:
twitter_archive['rating_denominator'].value_counts()

10     2333
11        3
50        3
80        2
20        2
2         1
16        1
40        1
70        1
15        1
90        1
110       1
120       1
130       1
150       1
170       1
7         1
0         1
Name: rating_denominator, dtype: int64

In [16]:
twitter_archive['rating_numerator'].value_counts()

12      558
11      464
10      461
13      351
9       158
8       102
7        55
14       54
5        37
6        32
3        19
4        17
1         9
2         9
420       2
0         2
15        2
75        2
80        1
20        1
24        1
26        1
44        1
50        1
60        1
165       1
84        1
88        1
144       1
182       1
143       1
666       1
960       1
1776      1
17        1
27        1
45        1
99        1
121       1
204       1
Name: rating_numerator, dtype: int64

In [17]:
#Analisando o dataframe tweet status
tweet_status.head(20)

,tweet_id,favorite_counts,retweet_counts
0,892420643555336193,38211,8371
1,892177421306343426,32758,6179
2,891815181378084864,24666,4091
3,891689557279858688,41542,8506
4,891327558926688256,39727,9212
5,891087950875897856,19939,3064
6,890971913173991426,11668,2033
7,890729181411237888,64480,18576
8,890609185150312448,27399,4206
9,890240255349198849,31452,7271


In [18]:
tweet_status.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2342 entries, 0 to 2341
Data columns (total 3 columns):
tweet_id           2342 non-null int64
favorite_counts    2342 non-null int64
retweet_counts     2342 non-null int64
dtypes: int64(3)
memory usage: 55.0 KB


### Arrumação

- As colunas `doggo`, `floofer`, `pupper` e `puppo` deveriam ser uma única coluna preenchida com sua respectiva classificação.
- A maior parte das informações estão no DataFrame `twitter_archive`, porém existem informações necessárias que estão no DataFrame `tweet_status`.

### Qualidade

- Como motivação do projeto, só é necessário classificações originais, que não sejam retweets, portanto, retirar a linhas que são retweets de acordo com a coluna `retweeted_status_id`.
- Alguns cachorros estão com o nome `a`, `an`.
- Alguns denominadores estão com valores diferentes de 10. Coluna `rating_denominator`
- Alguns numeradores estão com valores suspeitos. Por ser uma conta do Twitter que tem uma proposta de entretenimento através de classificar cachorros, podemos entender alguns valores acima de 10, no entanto temos alguns casos de 1 ou 2 ocorrências que parecem errados. Coluna `rating_numerator`.
- A coluna `timestamp` está com o datatype errado. Atualmente está como `string`.
- Valores em branco na coluna `expanded_urls`.
- As colunas `in_reply_to_status_id`, `in_reply_to_user_id`, estão com o datatype errado. Atualmente estão como `float`.
- As colunas `favorite_counts` e `retweet_counts` possuem uma linha em branco.
- As colunas `rating_numerator`, `favorite_counts` e `retweet_counts` estão com o datatype errado. Atualmente estão como `float`.
- A coluna `tweet_id` está com o datatype errado. Atualmente está como `integer`.




# Limpeza

### Arrumação

#### As colunas `doggo`, `floofer`, `pupper` e `puppo` deveriam ser uma única coluna preenchida com sua respectiva classificação.

#### Definição
Criar uma coluna chamada `classification` e preencher com os valores `doggo`, `floofer`, `pupper`, `puppo` ou `N/A`.

#### Código

In [19]:
#criar uma cópia do DataFrame para não comprometer o DataFrame original
df = twitter_archive.copy()

In [20]:
#Apagar os valores None 
df = df.replace('None', '')

In [21]:
#criar e preencher a coluna classification com as informações das colunas doggo, floofer, pupper e puppo
df['classification'] = (df['doggo'].fillna('') + df['floofer'].fillna('') + df['pupper'].fillna('') + df['puppo'].fillna('')).replace('', np.nan)

In [22]:
#Dropar todas as colunas e deixar somente a classification
df = df.drop(columns=['in_reply_to_status_id', 'in_reply_to_user_id', 'timestamp', 'source', 'text', 'retweeted_status_id', 'retweeted_status_user_id', 'retweeted_status_timestamp', 'expanded_urls', 'rating_numerator', 'rating_denominator', 'name', 'doggo','floofer', 'pupper', 'puppo'])

In [23]:
#Acrescentar a coluna classification no DataFrame twitter_archive e remover as colunas doggo, floofer, pupper e puppo
twitter_archive = pd.merge(twitter_archive, df, on= 'tweet_id')
twitter_archive = twitter_archive.drop(columns=['doggo', 'floofer', 'pupper', 'puppo'])

#### Validação

In [24]:
twitter_archive.head(10)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,classification
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,NaN
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,NaN
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,NaN
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,NaN
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,NaN
5,891087950875897856,NaN,NaN,2017-07-29 00:08:17 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here we have a majestic great white breaching ...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891087950...,13,10,None,NaN
6,890971913173991426,NaN,NaN,2017-07-28 16:27:12 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Meet Jax. He enjoys ice cream so much he gets ...,NaN,NaN,NaN,"https://gofundme.com/ydvmve-surgery-for-jax,ht...",13,10,Jax,NaN
7,890729181411237888,NaN,NaN,2017-07-28 00:22:40 +0000,"<a href=""http://twitter.com/download/iphone"" r...",When you watch your owner call another dog a g...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/890729181...,13,10,None,NaN
8,890609185150312448,NaN,NaN,2017-07-27 16:25:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Zoey. She doesn't want to be one of th...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/890609185...,13,10,Zoey,NaN
9,890240255349198849,NaN,NaN,2017-07-26 15:59:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Cassie. She is a college pup. Studying...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/890240255...,14,10,Cassie,doggo


#### A maior parte das informações estão no DataFrame `twitter_archive`, porém existem informações necessárias que estão no DataFrame `tweet_status`.

#### Definição
Realizar um `merge` entre os DataFrames `twitter_archive` e `tweet_status`.

#### Código

In [25]:
twitter_archive_master = pd.merge(twitter_archive, tweet_status, on=['tweet_id'], how='left')

#### Validação

In [26]:
twitter_archive_master.head()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,classification,favorite_counts,retweet_counts
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,NaN,38211.0,8371.0
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,NaN,32758.0,6179.0
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,NaN,24666.0,4091.0
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,NaN,41542.0,8506.0
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,NaN,39727.0,9212.0


### Qualidade

#### Como motivação do projeto, só é necessário classificações originais, que não sejam retweets, portanto, retirar a linhas que são retweets de acordo com a coluna `retweeted_status_id`.

#### Definição
Retirar usando `drop` as linhas cujo não estão com a coluna `retweeted_status_id` em branco. Pois o preenchimento dessa coluna significa que a linha é um retweet e não queremos essas informações. Depois retirar a coluna usando `drop`pois ela não agregará mais informações para esse estudo.

#### Código

In [27]:
#Desconsiderando as linhas que estão com as colunas referente a retweet preenchidas e removendo essas colunas do DataFrame.
twitter_archive_master = twitter_archive_master[twitter_archive_master.retweeted_status_id.isnull() == True]
twitter_archive_master = twitter_archive_master.drop(columns=['retweeted_status_id', 'retweeted_status_user_id', 'retweeted_status_timestamp'])

#### Validação

In [28]:
twitter_archive_master.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2175 entries, 0 to 2355
Data columns (total 13 columns):
tweet_id                 2175 non-null int64
in_reply_to_status_id    78 non-null float64
in_reply_to_user_id      78 non-null float64
timestamp                2175 non-null object
source                   2175 non-null object
text                     2175 non-null object
expanded_urls            2117 non-null object
rating_numerator         2175 non-null int64
rating_denominator       2175 non-null int64
name                     2175 non-null object
classification           344 non-null object
favorite_counts          2174 non-null float64
retweet_counts           2174 non-null float64
dtypes: float64(4), int64(3), object(6)
memory usage: 237.9+ KB


#### Alguns cachorros estão com o nome `None`, `a`, `an`.

#### Definição
Existem alguns cachorros com os nomes `a` e `an`. Verificar se o nome deles foi mencionado na coluna `text` ou até mesmo no site do Twitter, anotar os IDs dos Tweets e os nomes corretos e então usar o `replace` para o nome correto. Para os que não tiverem nome, deixar como `Null`.
Para os casos onde o nome está `None`, alterar para valor `nulo`.

#### Código

In [29]:
#Criar um dataframe para eu consultar a URL de cada Twitter e procurar pelo nome correto daqueles que estão com o nome a.
df3 = twitter_archive_master[twitter_archive_master.name == 'a']

In [30]:
#Para aparecer a URL no Notebook, fiz um strip para retirar a parte igual do endereço.
df3.expanded_urls.str.strip('https://twitter.com/dog_rates/status/')

56                             881536004380872706/video/1
649     792913359805018113/photo/1,https://twitter.com...
801     772581559778025472/photo/1,https://twitter.com...
1002    747885874273214464/photo/1,https://twitter.com...
1004                           747816857231626240/photo/1
1017    746872823977771008/photo/1,https://twitter.com...
1049                           743222593470234624/photo/1
1193                           717537687239008257/photo/1
1207                           715733265223708672/photo/1
1340                           704859558691414016/photo/1
1351                           704054845121142784/photo/1
1361    703079050210877440/photo/1,https://twitter.com...
1368    702539513671897089/photo/1,https://twitter.com...
1382                           700864154249383937/photo/1
1499    692187005137076224/photo/1,https://twitter.com...
1737                           679530280114372609/photo/1
1785                           677644091929329666/video/1
1853          

In [31]:
#Criar um dataframe para eu consultar a URL de cada Twitter e procurar pelo nome correto daqueles que estão com o nome an.
df4 = twitter_archive_master[twitter_archive_master.name == 'an']

In [32]:
#Para aparecer a URL no Notebook, fiz um strip para retirar a parte igual do endereço.
df4.expanded_urls.str.strip('https://twitter.com/dog_rates/status/')

1025    746369468511756288/photo/1
1362    703041949650034688/photo/1
2204    668636665813057536/photo/1
2333    666337882303524864/photo/1
2335    666287406224695296/photo/1
2349    666051853826850816/photo/1
Name: expanded_urls, dtype: object

In [33]:
#Realizar o replace com os nomes corretos.
twitter_archive_master['name'] = np.where(twitter_archive_master['tweet_id'] == 881536004380872706, 'Puffie', twitter_archive_master['name'])
twitter_archive_master['name'] = np.where(twitter_archive_master['tweet_id'] == 792913359805018113, 'Forrest', twitter_archive_master['name'])
twitter_archive_master['name'] = np.where(twitter_archive_master['tweet_id'] == 675706639471788032, 'Wylie', twitter_archive_master['name'])
twitter_archive_master['name'] = np.where(twitter_archive_master['tweet_id'] == 673636718965334016, 'Kip', twitter_archive_master['name'])
twitter_archive_master['name'] = np.where(twitter_archive_master['tweet_id'] == 671743150407421952, 'Jacob', twitter_archive_master['name'])
twitter_archive_master['name'] = np.where(twitter_archive_master['tweet_id'] == 671147085991960577, 'Rufus', twitter_archive_master['name'])
twitter_archive_master['name'] = np.where(twitter_archive_master['tweet_id'] == 670427002554466305, 'Spork', twitter_archive_master['name'])
twitter_archive_master['name'] = np.where(twitter_archive_master['tweet_id'] == 670361874861563904, 'Cherokee', twitter_archive_master['name'])
twitter_archive_master['name'] = np.where(twitter_archive_master['tweet_id'] == 670303360680108032, 'Hemry', twitter_archive_master['name'])
twitter_archive_master['name'] = np.where(twitter_archive_master['tweet_id'] == 669923323644657664, 'Alphred', twitter_archive_master['name'])
twitter_archive_master['name'] = np.where(twitter_archive_master['tweet_id'] == 669564461267722241, 'Alfredo', twitter_archive_master['name'])
twitter_archive_master['name'] = np.where(twitter_archive_master['tweet_id'] == 668955713004314625, 'Leroi', twitter_archive_master['name'])
twitter_archive_master['name'] = np.where(twitter_archive_master['tweet_id'] == 668507509523615744, 'Chuk', twitter_archive_master['name'])
twitter_archive_master['name'] = np.where(twitter_archive_master['tweet_id'] == 668171859951755264, 'Alfonso', twitter_archive_master['name'])
twitter_archive_master['name'] = np.where(twitter_archive_master['tweet_id'] == 667861340749471744, 'Cheryl', twitter_archive_master['name'])
twitter_archive_master['name'] = np.where(twitter_archive_master['tweet_id'] == 667773195014021121, 'Jessiga', twitter_archive_master['name'])
twitter_archive_master['name'] = np.where(twitter_archive_master['tweet_id'] == 667538891197542400, 'Klint', twitter_archive_master['name'])
twitter_archive_master['name'] = np.where(twitter_archive_master['tweet_id'] == 667470559035432960, 'Kohl', twitter_archive_master['name'])
twitter_archive_master['name'] = np.where(twitter_archive_master['tweet_id'] == 667177989038297088, 'Daryl', twitter_archive_master['name'])
twitter_archive_master['name'] = np.where(twitter_archive_master['tweet_id'] == 666983947667116034, 'Pepe', twitter_archive_master['name'])
twitter_archive_master['name'] = np.where(twitter_archive_master['tweet_id'] == 666781792255496192, 'Octaviath', twitter_archive_master['name'])
twitter_archive_master['name'] = np.where(twitter_archive_master['tweet_id'] == 666701168228331520, 'Johm', twitter_archive_master['name'])
twitter_archive_master['name'] = np.where(twitter_archive_master['tweet_id'] == 668636665813057536, 'Berta', twitter_archive_master['name'])

In [44]:
#Mudar os nomes None, a e an para nulo.
twitter_archive_master['name'] = twitter_archive_master.name.replace('None', '')
twitter_archive_master['name'] = twitter_archive_master.name.replace('a', '')
twitter_archive_master['name'] = twitter_archive_master.name.replace('an', '')

#### Validação

In [45]:
twitter_archive_master.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2175 entries, 0 to 2355
Data columns (total 13 columns):
tweet_id                 2175 non-null int64
in_reply_to_status_id    78 non-null float64
in_reply_to_user_id      78 non-null float64
timestamp                2175 non-null object
source                   2175 non-null object
text                     2175 non-null object
expanded_urls            2117 non-null object
rating_numerator         2175 non-null int64
rating_denominator       2175 non-null int64
name                     2175 non-null object
classification           344 non-null object
favorite_counts          2174 non-null float64
retweet_counts           2174 non-null float64
dtypes: float64(4), int64(3), object(6)
memory usage: 237.9+ KB


In [46]:
twitter_archive_master['name'].value_counts()

            718
Charlie      11
Lucy         11
Oliver       10
Cooper       10
Tucker        9
Penny         9
Sadie         8
the           8
Winston       8
Lola          8
Toby          7
Daisy         7
Oscar         6
Bo            6
Stanley       6
Koda          6
Bailey        6
Bella         6
Jax           6
Buddy         5
Leo           5
Milo          5
Rusty         5
Chester       5
Dave          5
Scout         5
Louis         5
Bentley       5
very          4
           ... 
Izzy          1
Fabio         1
Jett          1
Huxley        1
Leroi         1
Taz           1
Clarkus       1
Tessa         1
Odin          1
Sephie        1
Griswold      1
Nico          1
Chevy         1
Mojo          1
Randall       1
Charl         1
Luther        1
Doobert       1
Shikha        1
Alfredo       1
Apollo        1
Nugget        1
Buddah        1
Dug           1
Lili          1
Comet         1
Dewey         1
Kanu          1
Tino          1
Craig         1
Name: name, Length: 974,

#### Alguns denominadores estão com valores diferentes de 10. Coluna `rating_denominator`.

#### Definição
Alterar via `replace` o valor de todos que forem diferente de 10, para 10.

#### Código

In [47]:
twitter_archive_master[twitter_archive_master.rating_denominator != 10]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,classification,favorite_counts,retweet_counts
313,835246439529840640,8.352460e+17,2.625958e+07,2017-02-24 21:54:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@jonnysun @Lin_Manuel ok jomny I know you're e...,NaN,960,0,,NaN,2205.0,78.0
342,832088576586297345,8.320875e+17,3.058208e+07,2017-02-16 04:45:50 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@docmisterio account started on 11/15/15,NaN,11,15,,NaN,70.0,2.0
433,820690176645140481,NaN,NaN,2017-01-15 17:52:40 +0000,"<a href=""http://twitter.com/download/iphone"" r...",The floofs have been released I repeat the flo...,https://twitter.com/dog_rates/status/820690176...,84,70,,NaN,13029.0,3538.0
516,810984652412424192,NaN,NaN,2016-12-19 23:06:23 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Meet Sam. She smiles 24/7 &amp; secretly aspir...,"https://www.gofundme.com/sams-smile,https://tw...",24,7,Sam,NaN,5724.0,1579.0
902,758467244762497024,NaN,NaN,2016-07-28 01:00:57 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Why does this never happen at my front door......,https://twitter.com/dog_rates/status/758467244...,165,150,,NaN,5109.0,2422.0
1068,740373189193256964,NaN,NaN,2016-06-08 02:41:38 +0000,"<a href=""http://twitter.com/download/iphone"" r...","After so many requests, this is Bretagne. She ...",https://twitter.com/dog_rates/status/740373189...,9,11,,NaN,36636.0,14276.0
1120,731156023742988288,NaN,NaN,2016-05-13 16:15:54 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Say hello to this unbelievably well behaved sq...,https://twitter.com/dog_rates/status/731156023...,204,170,this,NaN,4029.0,1359.0
1165,722974582966214656,NaN,NaN,2016-04-21 02:25:47 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Happy 4/20 from the squad! 13/10 for all https...,https://twitter.com/dog_rates/status/722974582...,4,20,,NaN,4316.0,1679.0
1202,716439118184652801,NaN,NaN,2016-04-03 01:36:11 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Bluebert. He just saw that both #Final...,https://twitter.com/dog_rates/status/716439118...,50,50,Bluebert,NaN,2479.0,228.0
1228,713900603437621249,NaN,NaN,2016-03-27 01:29:02 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Happy Saturday here's 9 puppers on a bench. 99...,https://twitter.com/dog_rates/status/713900603...,99,90,,NaN,2971.0,793.0


In [48]:
twitter_archive_master['rating_denominator'].value_counts()

10     2153
50        3
80        2
11        2
20        2
2         1
16        1
40        1
70        1
15        1
90        1
110       1
120       1
130       1
150       1
170       1
7         1
0         1
Name: rating_denominator, dtype: int64

Após analisar os casos diferentes de 10, percebi que quando há mais de um cachorro na foto, a equipe do We Rate Dogs muda o denominador de acordo, 20 para 2 cachorros, 30 para 3 cachorros e assim por diante. Portanto, é necessário corrigir apenas os valores que não são múltiplos de 10.
No caso, 11, 2, 16, 15, 7 e 0.

In [49]:
#Realizar o replace com o denominador correto.
twitter_archive_master['rating_denominator'] = np.where(twitter_archive_master['rating_denominator'] == 11, 10, twitter_archive_master['rating_denominator'])
twitter_archive_master['rating_denominator'] = np.where(twitter_archive_master['rating_denominator'] == 2, 10, twitter_archive_master['rating_denominator'])
twitter_archive_master['rating_denominator'] = np.where(twitter_archive_master['rating_denominator'] == 16, 10, twitter_archive_master['rating_denominator'])
twitter_archive_master['rating_denominator'] = np.where(twitter_archive_master['rating_denominator'] == 15, 10, twitter_archive_master['rating_denominator'])
twitter_archive_master['rating_denominator'] = np.where(twitter_archive_master['rating_denominator'] == 7, 10, twitter_archive_master['rating_denominator'])
twitter_archive_master['rating_denominator'] = np.where(twitter_archive_master['rating_denominator'] == 0, 10, twitter_archive_master['rating_denominator'])


#### Validação

In [50]:
twitter_archive_master['rating_denominator'].value_counts()

10     2160
50        3
80        2
20        2
170       1
150       1
130       1
120       1
110       1
90        1
70        1
40        1
Name: rating_denominator, dtype: int64

#### Alguns numeradores estão com valores suspeitos. Por ser uma conta do Twitter que tem uma proposta de entretenimento através de classificar cachorros, podemos entender alguns valores acima de 10, no entanto temos alguns casos de 1 ou 2 ocorrências que parecem errados. Coluna `rating_numerator`.

#### Definição
Procurar na coluna `text` ou até mesmo no próprio site do Twitter o numerador correto e alterar via `replace`.

#### Código

In [51]:
twitter_archive_master['rating_numerator'].value_counts()

12      500
10      442
11      426
13      307
9       156
8        98
7        54
14       43
5        36
6        32
3        19
4        17
2         9
1         8
420       2
0         2
20        1
24        1
26        1
84        1
44        1
50        1
60        1
80        1
165       1
88        1
144       1
182       1
143       1
666       1
960       1
1776      1
15        1
17        1
27        1
45        1
75        1
99        1
121       1
204       1
Name: rating_numerator, dtype: int64

In [52]:
#Conferindo os valores diferentes.
twitter_archive_master[twitter_archive_master.rating_numerator ==  204]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,classification,favorite_counts,retweet_counts
1120,731156023742988288,NaN,NaN,2016-05-13 16:15:54 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Say hello to this unbelievably well behaved sq...,https://twitter.com/dog_rates/status/731156023...,204,170,this,NaN,4029.0,1359.0


In [53]:
twitter_archive_master[twitter_archive_master.rating_numerator ==  121]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,classification,favorite_counts,retweet_counts
1635,684222868335505415,NaN,NaN,2016-01-05 04:00:18 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Someone help the girl is being mugged. Several...,https://twitter.com/dog_rates/status/684222868...,121,110,,NaN,4060.0,1482.0


In [54]:
twitter_archive_master[twitter_archive_master.rating_numerator ==  99]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,classification,favorite_counts,retweet_counts
1228,713900603437621249,NaN,NaN,2016-03-27 01:29:02 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Happy Saturday here's 9 puppers on a bench. 99...,https://twitter.com/dog_rates/status/713900603...,99,90,,NaN,2971.0,793.0


In [55]:
twitter_archive_master[twitter_archive_master.rating_numerator ==  75]
#valor correto: 9,75

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,classification,favorite_counts,retweet_counts
695,786709082849828864,NaN,NaN,2016-10-13 23:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...","This is Logan, the Chow who lived. He solemnly...",https://twitter.com/dog_rates/status/786709082...,75,10,Logan,NaN,19550.0,6712.0


In [56]:
twitter_archive_master[twitter_archive_master.rating_numerator ==  45]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,classification,favorite_counts,retweet_counts
1274,709198395643068416,NaN,NaN,2016-03-14 02:04:08 +0000,"<a href=""http://twitter.com/download/iphone"" r...","From left to right:\nCletus, Jerome, Alejandro...",https://twitter.com/dog_rates/status/709198395...,45,50,,NaN,2537.0,685.0


In [57]:
twitter_archive_master[twitter_archive_master.rating_numerator ==  27]
#valor correto: 11.27

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,classification,favorite_counts,retweet_counts
763,778027034220126208,NaN,NaN,2016-09-20 00:24:34 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Sophie. She's a Jubilant Bush Pupper. ...,https://twitter.com/dog_rates/status/778027034...,27,10,Sophie,pupper,7023.0,1779.0


In [58]:
twitter_archive_master[twitter_archive_master.rating_numerator ==  17]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,classification,favorite_counts,retweet_counts
55,881633300179243008,8.816070e+17,47384430.0,2017-07-02 21:58:53 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@roushfenway These are good dogs but 17/10 is ...,NaN,17,10,,NaN,124.0,7.0


In [59]:
twitter_archive_master[twitter_archive_master.rating_numerator ==  15]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,classification,favorite_counts,retweet_counts
291,838085839343206401,8.380855e+17,2.894131e+09,2017-03-04 17:56:49 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@bragg6of8 @Andy_Pace_ we are still looking fo...,NaN,15,10,,NaN,149.0,0.0


In [60]:
twitter_archive_master[twitter_archive_master.rating_numerator ==  1776]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,classification,favorite_counts,retweet_counts
979,749981277374128128,NaN,NaN,2016-07-04 15:00:45 +0000,"<a href=""https://about.twitter.com/products/tw...",This is Atticus. He's quite simply America af....,https://twitter.com/dog_rates/status/749981277...,1776,10,Atticus,NaN,5415.0,2647.0


In [61]:
twitter_archive_master[twitter_archive_master.rating_numerator ==  960]
#Valor correto: 13

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,classification,favorite_counts,retweet_counts
313,835246439529840640,8.352460e+17,26259576.0,2017-02-24 21:54:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@jonnysun @Lin_Manuel ok jomny I know you're e...,NaN,960,10,,NaN,2205.0,78.0


In [62]:
twitter_archive_master[twitter_archive_master.rating_numerator ==  666]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,classification,favorite_counts,retweet_counts
189,855860136149123072,8.558585e+17,13615722.0,2017-04-22 19:05:32 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@s8n You tried very hard to portray this good ...,NaN,666,10,,NaN,5143.0,1067.0


In [63]:
twitter_archive_master[twitter_archive_master.rating_numerator ==  143]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,classification,favorite_counts,retweet_counts
1634,684225744407494656,6.842229e+17,4.196984e+09,2016-01-05 04:11:44 +0000,"<a href=""http://twitter.com/download/iphone"" r...","Two sneaky puppers were not initially seen, mo...",https://twitter.com/dog_rates/status/684225744...,143,130,,NaN,1307.0,229.0


In [64]:
twitter_archive_master[twitter_archive_master.rating_numerator ==  182]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,classification,favorite_counts,retweet_counts
290,838150277551247360,8.381455e+17,21955058.0,2017-03-04 22:12:52 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@markhoppus 182/10,NaN,182,10,,NaN,1768.0,354.0


In [65]:
twitter_archive_master[twitter_archive_master.rating_numerator ==  144]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,classification,favorite_counts,retweet_counts
1779,677716515794329600,NaN,NaN,2015-12-18 05:06:23 +0000,"<a href=""http://twitter.com/download/iphone"" r...",IT'S PUPPERGEDDON. Total of 144/120 ...I think...,https://twitter.com/dog_rates/status/677716515...,144,120,,NaN,3185.0,1031.0


In [66]:
twitter_archive_master[twitter_archive_master.rating_numerator ==  88]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,classification,favorite_counts,retweet_counts
1843,675853064436391936,NaN,NaN,2015-12-13 01:41:41 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here we have an entire platoon of puppers. Tot...,https://twitter.com/dog_rates/status/675853064...,88,80,,NaN,2784.0,1372.0


In [67]:
twitter_archive_master[twitter_archive_master.rating_numerator ==  165]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,classification,favorite_counts,retweet_counts
902,758467244762497024,NaN,NaN,2016-07-28 01:00:57 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Why does this never happen at my front door......,https://twitter.com/dog_rates/status/758467244...,165,150,,NaN,5109.0,2422.0


In [68]:
twitter_archive_master[twitter_archive_master.rating_numerator ==  80]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,classification,favorite_counts,retweet_counts
1254,710658690886586372,NaN,NaN,2016-03-18 02:46:49 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here's a brigade of puppers. All look very pre...,https://twitter.com/dog_rates/status/710658690...,80,80,,NaN,2425.0,604.0


In [69]:
twitter_archive_master[twitter_archive_master.rating_numerator ==  60]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,classification,favorite_counts,retweet_counts
1351,704054845121142784,NaN,NaN,2016-02-28 21:25:30 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here is a whole flock of puppers. 60/50 I'll ...,https://twitter.com/dog_rates/status/704054845...,60,50,,NaN,3073.0,979.0


In [70]:
twitter_archive_master[twitter_archive_master.rating_numerator ==  50]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,classification,favorite_counts,retweet_counts
1202,716439118184652801,NaN,NaN,2016-04-03 01:36:11 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Bluebert. He just saw that both #Final...,https://twitter.com/dog_rates/status/716439118...,50,50,Bluebert,NaN,2479.0,228.0


In [71]:
twitter_archive_master[twitter_archive_master.rating_numerator ==  84]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,classification,favorite_counts,retweet_counts
433,820690176645140481,NaN,NaN,2017-01-15 17:52:40 +0000,"<a href=""http://twitter.com/download/iphone"" r...",The floofs have been released I repeat the flo...,https://twitter.com/dog_rates/status/820690176...,84,70,,NaN,13029.0,3538.0


In [72]:
twitter_archive_master[twitter_archive_master.rating_numerator ==  44]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,classification,favorite_counts,retweet_counts
1433,697463031882764288,NaN,NaN,2016-02-10 16:51:59 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Happy Wednesday here's a bucket of pups. 44/40...,https://twitter.com/dog_rates/status/697463031...,44,40,,NaN,3597.0,1475.0


In [73]:
twitter_archive_master[twitter_archive_master.rating_numerator ==  26]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,classification,favorite_counts,retweet_counts
1712,680494726643068929,NaN,NaN,2015-12-25 21:06:00 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here we have uncovered an entire battalion of ...,https://twitter.com/dog_rates/status/680494726...,26,10,,NaN,1800.0,519.0


In [74]:
twitter_archive_master[twitter_archive_master.rating_numerator ==  24]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,classification,favorite_counts,retweet_counts
516,810984652412424192,NaN,NaN,2016-12-19 23:06:23 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Meet Sam. She smiles 24/7 &amp; secretly aspir...,"https://www.gofundme.com/sams-smile,https://tw...",24,10,Sam,NaN,5724.0,1579.0


In [75]:
twitter_archive_master[twitter_archive_master.rating_numerator ==  20]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,classification,favorite_counts,retweet_counts
1663,682808988178739200,6.827884e+17,4.196984e+09,2016-01-01 06:22:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...","I'm aware that I could've said 20/16, but here...",NaN,20,10,,NaN,1869.0,193.0


In [76]:
twitter_archive_master[twitter_archive_master.rating_numerator ==  0]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,classification,favorite_counts,retweet_counts
315,835152434251116546,NaN,NaN,2017-02-24 15:40:31 +0000,"<a href=""http://twitter.com/download/iphone"" r...",When you're so blinded by your systematic plag...,https://twitter.com/dog_rates/status/835152434...,0,10,,NaN,23695.0,3258.0
1016,746906459439529985,7.468859e+17,4.196984e+09,2016-06-26 03:22:31 +0000,"<a href=""http://twitter.com/download/iphone"" r...","PUPDATE: can't see any. Even if I could, I cou...",https://twitter.com/dog_rates/status/746906459...,0,10,,NaN,3051.0,318.0


In [77]:
twitter_archive_master[twitter_archive_master.rating_numerator ==  420]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,classification,favorite_counts,retweet_counts
188,855862651834028034,8.558616e+17,194351775.0,2017-04-22 19:15:32 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@dhmontgomery We also gave snoop dogg a 420/10...,NaN,420,10,,NaN,352.0,28.0
2074,670842764863651840,NaN,NaN,2015-11-29 05:52:33 +0000,"<a href=""http://twitter.com/download/iphone"" r...",After so many requests... here you go.\n\nGood...,https://twitter.com/dog_rates/status/670842764...,420,10,,NaN,25061.0,9031.0


In [78]:
twitter_archive_master[twitter_archive_master.rating_numerator ==  1]
#Id 666287406224695296, valor correto: 9

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,classification,favorite_counts,retweet_counts
1446,696490539101908992,6.964887e+17,4.196984e+09,2016-02-08 00:27:39 +0000,"<a href=""http://twitter.com/download/iphone"" r...",After reading the comments I may have overesti...,NaN,1,10,,NaN,1288.0,156.0
1869,675153376133427200,NaN,NaN,2015-12-11 03:21:23 +0000,"<a href=""http://twitter.com/download/iphone"" r...",What kind of person sends in a picture without...,https://twitter.com/dog_rates/status/675153376...,1,10,,NaN,6490.0,2744.0
1940,673716320723169284,6.737159e+17,4.196984e+09,2015-12-07 04:11:02 +0000,"<a href=""http://twitter.com/download/iphone"" r...",The millennials have spoken and we've decided ...,NaN,1,10,,NaN,3147.0,792.0
2038,671550332464455680,6.715449e+17,4.196984e+09,2015-12-01 04:44:10 +0000,"<a href=""http://twitter.com/download/iphone"" r...",After 22 minutes of careful deliberation this ...,NaN,1,10,,NaN,936.0,213.0
2091,670783437142401025,NaN,NaN,2015-11-29 01:56:48 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Flamboyant pup here. Probably poisonous. Won't...,https://twitter.com/dog_rates/status/670783437...,1,10,,NaN,833.0,408.0
2261,667549055577362432,NaN,NaN,2015-11-20 03:44:31 +0000,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",Never seen dog like this. Breathes heavy. Tilt...,https://twitter.com/dog_rates/status/667549055...,1,10,,NaN,5875.0,2334.0
2335,666287406224695296,NaN,NaN,2015-11-16 16:11:11 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is an Albanian 3 1/2 legged Episcopalian...,https://twitter.com/dog_rates/status/666287406...,1,10,,NaN,146.0,64.0
2338,666104133288665088,NaN,NaN,2015-11-16 04:02:55 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Not familiar with this breed. No tail (weird)....,https://twitter.com/dog_rates/status/666104133...,1,10,,NaN,14163.0,6485.0


In [79]:
twitter_archive_master[twitter_archive_master.rating_numerator ==  2]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,classification,favorite_counts,retweet_counts
1761,678675843183484930,NaN,NaN,2015-12-20 20:38:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Exotic pup here. Tail long af. Throat looks sw...,https://twitter.com/dog_rates/status/678675843...,2,10,,NaN,3007.0,1584.0
1764,678424312106393600,NaN,NaN,2015-12-20 03:58:55 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Crystal. She's a shitty fireman. No se...,https://twitter.com/dog_rates/status/678424312...,2,10,Crystal,NaN,5663.0,2718.0
1920,674265582246694913,NaN,NaN,2015-12-08 16:33:36 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Henry. He's a shit dog. Short pointy e...,https://twitter.com/dog_rates/status/674265582...,2,10,Henry,NaN,1682.0,887.0
2079,670826280409919488,NaN,NaN,2015-11-29 04:47:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Scary dog here. Too many legs. Extra tail. Not...,https://twitter.com/dog_rates/status/670826280...,2,10,,NaN,5666.0,4194.0
2237,668142349051129856,NaN,NaN,2015-11-21 19:02:04 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This lil pup is Oliver. Hops around. Has wings...,https://twitter.com/dog_rates/status/668142349...,2,10,,NaN,575.0,295.0
2246,667878741721415682,NaN,NaN,2015-11-21 01:34:35 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tedrick. He lives on the edge. Needs s...,https://twitter.com/dog_rates/status/667878741...,2,10,Tedrick,NaN,399.0,122.0
2310,666786068205871104,NaN,NaN,2015-11-18 01:12:41 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Unfamiliar with this breed. Ears pointy af. Wo...,https://twitter.com/dog_rates/status/666786068...,2,10,,NaN,764.0,488.0
2326,666411507551481857,NaN,NaN,2015-11-17 00:24:19 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is quite the dog. Gets really excited whe...,https://twitter.com/dog_rates/status/666411507...,2,10,quite,NaN,439.0,323.0
2349,666051853826850816,NaN,NaN,2015-11-16 00:35:11 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is an odd dog. Hard on the outside but lo...,https://twitter.com/dog_rates/status/666051853...,2,10,,NaN,1208.0,841.0


In [80]:
twitter_archive_master['rating_numerator'] = np.where(twitter_archive_master['tweet_id'] == 786709082849828864, 9, twitter_archive_master['rating_numerator'])
twitter_archive_master['rating_numerator'] = np.where(twitter_archive_master['tweet_id'] == 778027034220126208, 11, twitter_archive_master['rating_numerator'])
twitter_archive_master['rating_numerator'] = np.where(twitter_archive_master['tweet_id'] == 835246439529840640, 13, twitter_archive_master['rating_numerator'])
twitter_archive_master['rating_numerator'] = np.where(twitter_archive_master['tweet_id'] == 666287406224695296, 9, twitter_archive_master['rating_numerator'])

#### Validação

In [81]:
twitter_archive_master['rating_numerator'].value_counts()

12      500
10      442
11      427
13      308
9       158
8        98
7        54
14       43
5        36
6        32
3        19
4        17
2         9
1         7
420       2
0         2
60        1
20        1
24        1
26        1
44        1
50        1
165       1
80        1
84        1
88        1
144       1
143       1
204       1
666       1
1776      1
15        1
17        1
45        1
99        1
121       1
182       1
Name: rating_numerator, dtype: int64

#### A coluna`timestamp` está com o datatype errado. Atualmente está como `string`.

#### Definição
Alterar a coluna para o datatype `datetime`.

#### Código

In [82]:
#Alterando o datatype da coluna timestamp.
twitter_archive_master.timestamp = pd.to_datetime(twitter_archive_master.timestamp)

#### Validação

In [83]:
twitter_archive_master.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2175 entries, 0 to 2355
Data columns (total 13 columns):
tweet_id                 2175 non-null int64
in_reply_to_status_id    78 non-null float64
in_reply_to_user_id      78 non-null float64
timestamp                2175 non-null datetime64[ns]
source                   2175 non-null object
text                     2175 non-null object
expanded_urls            2117 non-null object
rating_numerator         2175 non-null int64
rating_denominator       2175 non-null int64
name                     2175 non-null object
classification           344 non-null object
favorite_counts          2174 non-null float64
retweet_counts           2174 non-null float64
dtypes: datetime64[ns](1), float64(4), int64(3), object(5)
memory usage: 237.9+ KB


#### Valores em branco na coluna `expanded_urls`.

#### Definição
Colocar o endereço correto: https://twitter.com/dog_rates/status/ + tweet_id

#### Código

In [87]:
#Checar linhas com a coluna expanded_urls em branco
twitter_archive_master[twitter_archive_master.expanded_urls.isnull() == True]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,classification,favorite_counts,retweet_counts
30,886267009285017600,8.862664e+17,2.281182e+09,2017-07-15 16:51:35,"<a href=""http://twitter.com/download/iphone"" r...",@NonWhiteHat @MayhewMayhem omg hello tanner yo...,NaN,12,10,,NaN,116.0,4.0
55,881633300179243008,8.816070e+17,4.738443e+07,2017-07-02 21:58:53,"<a href=""http://twitter.com/download/iphone"" r...",@roushfenway These are good dogs but 17/10 is ...,NaN,17,10,,NaN,124.0,7.0
64,879674319642796034,8.795538e+17,3.105441e+09,2017-06-27 12:14:36,"<a href=""http://twitter.com/download/iphone"" r...",@RealKentMurphy 14/10 confirmed,NaN,14,10,,NaN,308.0,10.0
113,870726314365509632,8.707262e+17,1.648776e+07,2017-06-02 19:38:25,"<a href=""http://twitter.com/download/iphone"" r...",@ComplicitOwl @ShopWeRateDogs &gt;10/10 is res...,NaN,10,10,,NaN,119.0,3.0
148,863427515083354112,8.634256e+17,7.759620e+07,2017-05-13 16:15:35,"<a href=""http://twitter.com/download/iphone"" r...",@Jack_Septic_Eye I'd need a few more pics to p...,NaN,12,10,,NaN,2280.0,99.0
179,857214891891077121,8.571567e+17,1.806710e+08,2017-04-26 12:48:51,"<a href=""http://twitter.com/download/iphone"" r...",@Marc_IRL pixelated af 12/10,NaN,12,10,,NaN,232.0,18.0
186,856288084350160898,8.562860e+17,2.792810e+08,2017-04-23 23:26:03,"<a href=""http://twitter.com/download/iphone"" r...",@xianmcguire @Jenna_Marbles Kardashians wouldn...,NaN,14,10,,NaN,527.0,17.0
188,855862651834028034,8.558616e+17,1.943518e+08,2017-04-22 19:15:32,"<a href=""http://twitter.com/download/iphone"" r...",@dhmontgomery We also gave snoop dogg a 420/10...,NaN,420,10,,NaN,352.0,28.0
189,855860136149123072,8.558585e+17,1.361572e+07,2017-04-22 19:05:32,"<a href=""http://twitter.com/download/iphone"" r...",@s8n You tried very hard to portray this good ...,NaN,666,10,,NaN,5143.0,1067.0
218,850333567704068097,8.503288e+17,2.195506e+07,2017-04-07 13:04:55,"<a href=""http://twitter.com/download/iphone"" r...",@markhoppus MARK THAT DOG HAS SEEN AND EXPERIE...,NaN,13,10,,NaN,3550.0,352.0


In [88]:
#Preencher a coluna expanded_urls com o link correto.
#

for i in twitter_archive_master:
    twitter_archive_master['expanded_urls'] = np.where(twitter_archive_master['expanded_urls'].isnull(), ('https://twitter.com/dog_rates/status/' + str(twitter_archive_master.tweet_id)), twitter_archive_master['expanded_urls'])   

#### Validação

In [89]:
twitter_archive_master.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2175 entries, 0 to 2355
Data columns (total 13 columns):
tweet_id                 2175 non-null int64
in_reply_to_status_id    78 non-null float64
in_reply_to_user_id      78 non-null float64
timestamp                2175 non-null datetime64[ns]
source                   2175 non-null object
text                     2175 non-null object
expanded_urls            2175 non-null object
rating_numerator         2175 non-null int64
rating_denominator       2175 non-null int64
name                     2175 non-null object
classification           344 non-null object
favorite_counts          2174 non-null float64
retweet_counts           2174 non-null float64
dtypes: datetime64[ns](1), float64(4), int64(3), object(5)
memory usage: 237.9+ KB


#### As colunas `in_reply_to_status_id`, `in_reply_to_user_id`, estão com o datatype errado. Atualmente estão como `float`.

#### Definição
Alterar as colunas para o datatype `object`.

#### Código

In [90]:
#Alterando o datatype das colunas in_reply_to_status_id e in_reply_to_user_id
twitter_archive_master.in_reply_to_status_id = twitter_archive_master.in_reply_to_status_id.astype('object')
twitter_archive_master.in_reply_to_user_id = twitter_archive_master.in_reply_to_user_id.astype('object')

#### Validação

In [91]:
twitter_archive_master.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2175 entries, 0 to 2355
Data columns (total 13 columns):
tweet_id                 2175 non-null int64
in_reply_to_status_id    78 non-null object
in_reply_to_user_id      78 non-null object
timestamp                2175 non-null datetime64[ns]
source                   2175 non-null object
text                     2175 non-null object
expanded_urls            2175 non-null object
rating_numerator         2175 non-null int64
rating_denominator       2175 non-null int64
name                     2175 non-null object
classification           344 non-null object
favorite_counts          2174 non-null float64
retweet_counts           2174 non-null float64
dtypes: datetime64[ns](1), float64(2), int64(3), object(7)
memory usage: 237.9+ KB


#### A coluna `tweet_id` está com o datatype errado. Atualmente está como `integer`.

#### Definição

Alterar as colunas para o datatype `object`.

#### Código

In [92]:
#Alterando o datatype da coluna tweet_id
twitter_archive_master.tweet_id = twitter_archive_master.tweet_id.astype('object')

#### Validação

In [93]:
twitter_archive_master.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2175 entries, 0 to 2355
Data columns (total 13 columns):
tweet_id                 2175 non-null object
in_reply_to_status_id    78 non-null object
in_reply_to_user_id      78 non-null object
timestamp                2175 non-null datetime64[ns]
source                   2175 non-null object
text                     2175 non-null object
expanded_urls            2175 non-null object
rating_numerator         2175 non-null int64
rating_denominator       2175 non-null int64
name                     2175 non-null object
classification           344 non-null object
favorite_counts          2174 non-null float64
retweet_counts           2174 non-null float64
dtypes: datetime64[ns](1), float64(2), int64(2), object(8)
memory usage: 237.9+ KB


#### As colunas `favorite_counts` e `retweet_counts` possuem uma linha em branco.

#### Definição

Como é necessário que a coluna seja `integer`, não podemos ter linhas com valores em branco. Irei dar um replace pra 0.

#### Código

In [94]:
#Preencher as linhas em branco com 0.
twitter_archive_master[['favorite_counts', 'retweet_counts']] = twitter_archive_master[['favorite_counts', 'retweet_counts']].fillna(0)


#### Validação

In [95]:
twitter_archive_master.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2175 entries, 0 to 2355
Data columns (total 13 columns):
tweet_id                 2175 non-null object
in_reply_to_status_id    78 non-null object
in_reply_to_user_id      78 non-null object
timestamp                2175 non-null datetime64[ns]
source                   2175 non-null object
text                     2175 non-null object
expanded_urls            2175 non-null object
rating_numerator         2175 non-null int64
rating_denominator       2175 non-null int64
name                     2175 non-null object
classification           344 non-null object
favorite_counts          2175 non-null float64
retweet_counts           2175 non-null float64
dtypes: datetime64[ns](1), float64(2), int64(2), object(8)
memory usage: 237.9+ KB


#### As colunas `rating_numerator`, `favorite_counts` e `reteweet_counts` estão como float.

#### Definição

Alterar as colunas para `integer`.

#### Código

In [96]:
#Alterando o datatype das colunas rating_numerator, favorite_counts e reteweet_counts
twitter_archive_master.rating_numerator = twitter_archive_master.rating_numerator.astype('int64')
twitter_archive_master.favorite_counts = twitter_archive_master.favorite_counts.astype('int64')
twitter_archive_master.retweet_counts = twitter_archive_master.retweet_counts.astype('int64')

#### Validação

In [97]:
twitter_archive_master.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2175 entries, 0 to 2355
Data columns (total 13 columns):
tweet_id                 2175 non-null object
in_reply_to_status_id    78 non-null object
in_reply_to_user_id      78 non-null object
timestamp                2175 non-null datetime64[ns]
source                   2175 non-null object
text                     2175 non-null object
expanded_urls            2175 non-null object
rating_numerator         2175 non-null int64
rating_denominator       2175 non-null int64
name                     2175 non-null object
classification           344 non-null object
favorite_counts          2175 non-null int64
retweet_counts           2175 non-null int64
dtypes: datetime64[ns](1), int64(4), object(8)
memory usage: 237.9+ KB


In [98]:
#Criar um CSV a partir do DataFrame twitter_archive_master.
twitter_archive_master.to_csv('twitter_archive_master.csv')